In [1]:
import numpy as np
from netline import Netline
import gym

In [2]:
env = gym.make('CartPole-v0')

In [3]:
weights = np.random.random((10, 10))
model = Netline(10, weights, num_inputs=4, num_outputs=2, learning_rate=0.001)

In [4]:
np.linalg.norm(model.W)

3.654981573307021

In [ ]:
epoch = 0
best_run = 0
last_runs = [] 
while True:
    model.reset()
    model.reset_rewards()
    initial = env.reset()[2:]
    initial[0] = (initial[0] + 0.42)/0.84
    initial[1] = (initial[1] + 12.0)/24.0
    input = []
    if initial[0] < 0.5:
        input += [initial[0], 0]
    else:
        input += [0, initial[0]]
    if initial[1] < 0.5:
        input += [initial[1], 0]
    else:
        input += [0, initial[1]]
    action = model.feed(input)
    run_length = 0
    while True:
        env.render()
        if action[0] - action[1] > 0.01:
            ret = env.step(0)
        elif action[0] - action[1] < -0.01:
            ret = env.step(1)
        else:
            ret = env.step(np.random.randint(2))
        next_position = ret[0][2:]
        r = next_position[0]**2 - next_position[1]**2
        model.reward(r)
        next_position[0] = (next_position[0] + 0.42)/0.84
        next_position[1] = (next_position[1] + 6.0)/12.0
        input = []
        if next_position[0] < 0.5:
            input += [next_position[0], 0]
        else:
            input += [0, next_position[0]]
        if next_position[1] < 0.5:
            input += [next_position[1], 0]
        else:
            input += [0, next_position[1]]
        action = model.feed(input)
        if ret[2]:
            break
        run_length += 1
    last_runs += [run_length]
    if len(last_runs) > 20:
        last_runs = last_runs[1:]
    avg_run_len = sum(last_runs)/20
    epoch += 1
    if run_length > best_run:
        best_run = run_length
    print('Run %d, Run length %d, Best run %d, Avg run %f, W norm: %f' % (epoch, run_length, best_run, avg_run_len, np.linalg.norm(model.W)), end=' '*200 + '\r')